# IMPORT THE LIBRARIES

In [349]:
#exc1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
from datetime import timedelta
import re

# LOAD THE PI DATA and LABORATORY ANALYSIS RESULTS

In [350]:
#Sheet From Pi
#Step1: Load the Data for the month
df_JAN2020Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\Pressure Filter Inv_2019_2.xlsx', sheet_name='PiData_PressureFilter', index_col=False)

#Drop the row[1]
df_JAN2020Pi = df_JAN2020Pi.drop(df_JAN2020Pi.index[0])

#Step2: Parse the DATETIME column
df_JAN2020Pi['DATETIME'] = df_JAN2020Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_JAN2020Pi.set_index('DATETIME', inplace=True)
df_comb = df_JAN2020Pi



In [351]:

import openpyxl
wb1 = openpyxl.load_workbook(r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\12.) December_ 2019.xlsx')

FT02wb = wb1['106FT02AB'] #####This block of code appends the Analysis Results

currentmonthyr = datetime.date(2019,12,1) #update for the targetmonth

print('Done', datetime.datetime.now())
print('Done. Excel PiData files loaded as dataframes. ', datetime.datetime.now())

Done 2020-02-03 16:10:36.761429
Done. Excel PiData files loaded as dataframes.  2020-02-03 16:10:36.761429



# DECLARE THE FUNCTIONS

In [352]:
#---fcn defn--start
#This function is used to replace the 'No good/ not enough data for calculation' value in the dataframe for data from PiData

def det_no_data(value):
    if type(value) == str:
        reg = re.search(r'\bCalculation\b', value)
        return reg[0]
    else:
        return value
#------fcn defn end

#fcn definition        
#used in the laboratory analysis results data

def filter_detrmin(cell_addrs):
    PF_regex2 = re.compile(r'[a-z0-8]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    filtername2 = PF_regex2.findall(str(cell_addrs))
    for i in range(0, len(filtername2)):
        if filtername2[i] != filtername2[-1]:
            if filtername2[i]+filtername2[i+1] == '2A':
                PFname = 'A'
                return PFname
            if filtername2[i]+filtername2[i+1] == '2B':
                PFname = 'B'
                return PFname      
#-----------------end of function defn-----------        

#fcn dfn removes the last character in a string if not an integer. does nothing otherwise
def check_ending_char(str_var):
    try:
        if type(int(str_var[-1])) != int:
                c = str_var[0:len(str_var)-1]
        else:
            c = str_var
    except ValueError:
        c = str_var[0:len(str_var)-1]
    return c

#fcn dfn removes the starting character in a string if not an integer. does nothing otherwise
def check_starting_char(str_var):
    try:
        if type(int(str_var[0])) != int:
            c = str_var[1:]
        else:
            c = str_var
    except ValueError:
        c = str_var[1:]
    return c

#fcn dfn
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact[0:5])
    f_time = check_ending_char(newtime)
    f_time1 = check_starting_char(f_time)
    t = parse(f_time1)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time


#fcn for formatting the dates from range str
def datesplitter_start(x):
    return parse(x.split('to')[0])
def datesplitter_end(x):
    return parse(x.split('to')[1])

#------------------

def remove_no_good_data_N2Dry_Feed(df): #removes the string value and fills it with the last good value
    
    df = pd.to_numeric(df, errors='coerce')
    for x in range(0, len(df)):
        if type(df[x]) == str:
            df[x] = df[x-1]
    df.fillna(method='ffill', inplace=True)
    return df

#---------------------------
def replace_no_good_data_with_LastGoodValues(df_col):
    
    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]== str):
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = np.nan

    df_col.fillna(method='ffill', inplace= True)
    return df_col
#--------------------------------------------
def replace_no_good_data_with_Zero(df_col):

    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]) == str:
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = 0
    df_col.fillna(value=0, inplace=True)
    
    return df_col
#--------------------------------------------
#FILTRATION TIME function
#--------------------------------------------
def generate_DF_comm(df_coltitle, str_colname):
        
    A_Filt_MaxList= list()
    A_Filt_MaxList[:] = []
    A_feedt_Val_List = list()
    A_feedt_Val_List[:] = []
    A_feedt_index_List = list()
    A_feedt_index_List[:] = []
    A_Pair_Filt_List = list()
    A_Pair_Filt_List[:] = []
    try:         
        for i in range(0, len(keyslist)):
            A_feedt_Val_List = list()
            A_feedt_Val_List[:] = []
            for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
                A_feedt = df_coltitle[j]
                A_feedt_Val_List.append(A_feedt)
            A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
            A_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
        A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
    except:
        pass
    df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
    df_A_Pair_Filt_t.columns = [str_colname, 'DATETIME']
    df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)  
    return df_A_Pair_Filt_t
#--------------------------------------------
#finding peaks FT for A_filtration time
#---function start---------
def getTheLocalMinMax(df_col1, df_col2):
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    b = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
   
    df_col2 = pd.to_numeric(df_col2, errors='coerce')
    f = (np.diff(np.sign(np.diff(df_col2))) < 0).nonzero()[0] + 1         # local max

    try:
        ft_cyc_lim = {}
        for i in range(0, len(b)):
            for j in range(0, len(f)):
                if f[j] > b[i] and f[j] < b[i+1]:
                    if f[j+1]<b[i+1]:
                        cyc_start = b[i]
                        cyc_end = f[j+1]
                        ft_cyc_lim.setdefault(cyc_start,cyc_end)
    except IndexError:
        print('\nDone')
    return ft_cyc_lim

#end of function----------------
#code for getting the CWP for CakeWash1 and CakeWash2

#----------------------------------------------------
def generate_DF_CW1(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslist)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))

            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
            A_CWP_1_inner.append(A_CWP_1)               

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------

#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_DF_CW2(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslist)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))

            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
            A_CWP_1_inner.append(A_CWP_1)               

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------
#---function to generate N2DRY DF
def generate_DF_for_N2DRY(df_colN2, str_colname):
    
    A_Pair_N2_Dry_List = list()
    A_Pair_N2_Dry_List[:] = []
    A_t_N2_Dry_indexList = list()
    A_t_N2_Dry_indexList[:] = []
    A_t_N2_Dry_val = list()
    A_t_N2_Dry_val[:] = []
    
    try:         
        for i in range(0, len(keyslist)):   
            A_t_N2_Dry = df_colN2[ft_cyc_lim[keyslist[i]]]
            A_t_N2_Dry_val.append(A_t_N2_Dry)
            A_t_N2_Dry_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]]))) 
    except:
        pass
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))
    df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
    df_A_Pair_N2DRY.columns = [str_colname, 'DATETIME']
    df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_N2DRY

#------------------------------------------------
#Functions for BRAVO FILTER Parameters

def generate_dfB_comm(df_coltitle, str_colname):
        
    A_Filt_MaxList= list()
    A_Filt_MaxList[:] = []
    A_feedt_Val_List = list()
    A_feedt_Val_List[:] = []
    A_feedt_index_List = list()
    A_feedt_index_List[:] = []
    A_Pair_Filt_List = list()
    A_Pair_Filt_List[:] = []
    try:         
        for i in range(0, len(keyslistB)):
            A_feedt_Val_List = list()
            A_feedt_Val_List[:] = []
            for j in range(keyslistB[i], ft_cyc_limB[keyslistB[i]]):
                A_feedt = df_coltitle[j]
                A_feedt_Val_List.append(A_feedt)
            A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
            A_feedt_index_List.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))  
        A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
    except:
        pass
    df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
    df_A_Pair_Filt_t.columns = [str_colname, 'DATETIME']
    df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)  
    return df_A_Pair_Filt_t
#--------------------------------------------


#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_dfB_CW1(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslistB)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslistB[i],ft_cyc_limB[keyslistB[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslistB[i]:ft_cyc_limB[keyslistB[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))

            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
            A_CWP_1_inner.append(A_CWP_1)               

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------

#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_dfB_CW2(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslistB)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslistB[i],ft_cyc_limB[keyslistB[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslistB[i]:ft_cyc_limB[keyslistB[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))

            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
            A_CWP_1_inner.append(A_CWP_1)               

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------
#---function to generate N2DRY DF
def generate_dfB_for_N2DRY(df_colN2, str_colname):
    
    A_Pair_N2_Dry_List = list()
    A_Pair_N2_Dry_List[:] = []
    A_t_N2_Dry_indexList = list()
    A_t_N2_Dry_indexList[:] = []
    A_t_N2_Dry_val = list()
    A_t_N2_Dry_val[:] = []
    
    try:         
        for i in range(0, len(keyslistB)):   
            A_t_N2_Dry = df_colN2[ft_cyc_limB[keyslistB[i]]]
            A_t_N2_Dry_val.append(A_t_N2_Dry)
            A_t_N2_Dry_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]]))) 
    except:
        pass
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))
    df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
    df_A_Pair_N2DRY.columns = [str_colname, 'DATETIME']
    df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_N2DRY

#------------------------------------------------

print('all functions declared', datetime.datetime.now())

all functions declared 2020-02-03 16:10:36.831153


# Clean the Data Using the functions

In [353]:
#Remove the no good data for t_FEED and N2_DRY ------------------

df_comb['A_t_FEED'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_FEED'])
df_comb['A_t_DRY'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_DRY'])

#Clean data from no good values
df_comb['A_t_CWSH1'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_CWSH1'])
df_comb['A_t_CWSH2'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_CWSH2'])
df_comb['A_t_PRESS1'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_PRESS1'])
df_comb['A_t_PRESS2'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_PRESS2'])

#Clean data from no good values
df_comb['106FT02A_MANFP'] = replace_no_good_data_with_Zero(df_comb['106FT02A_MANFP'])
df_comb['106FT02A_CWP'] = replace_no_good_data_with_Zero(df_comb['106FT02A_CWP'])
df_comb['106PU26A_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU26A_CUR'])
df_comb['106PU27A_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU27A_CUR'])
df_comb['PU26A_OP'] = replace_no_good_data_with_Zero(df_comb['PU26A_OP'])
df_comb['PU27A_OP'] = replace_no_good_data_with_Zero(df_comb['PU27A_OP'])



#Remove the no good data for t_FEED and N2_DRY for B dataframe------------------

df_comb['B_t_FEED'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_FEED'])
df_comb['B_t_DRY'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_DRY'])

#Clean data from no good values
df_comb['B_t_CWSH1'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_CWSH1'])
df_comb['B_t_CWSH2'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_CWSH2'])
df_comb['B_t_PRESS1'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_PRESS1'])
df_comb['B_t_PRESS2'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_PRESS2'])

#Clean data from no good values
df_comb['FT02B_MANFP'] = replace_no_good_data_with_Zero(df_comb['FT02B_MANFP'])
df_comb['FT02B_CWP'] = replace_no_good_data_with_Zero(df_comb['FT02B_CWP'])
df_comb['106PU26B_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU26B_CUR'])
df_comb['106PU27B_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU27B_CUR'])
df_comb['PU26B_OP'] = replace_no_good_data_with_Zero(df_comb['PU26B_OP'])
df_comb['PU27B_OP'] = replace_no_good_data_with_Zero(df_comb['PU27B_OP'])

print('Done. All param_df_generated', datetime.datetime.now())
#------------------------------------------------

Done. All param_df_generated 2020-02-03 16:11:11.778848


# SELECT THE ALPHA PARAMETERS

In [354]:
#Get the local min, max. Then generate the keys
ft_cyc_lim = getTheLocalMinMax(df_comb['A_t_FEED'], df_comb['A_t_DRY'])
keyslist = list(ft_cyc_lim.keys())
#----
print('Done. Filtration Cycle Time generated.', datetime.datetime.now())


Done
Done. Filtration Cycle Time generated. 2020-02-03 16:11:12.161813


In [355]:
#Generate the dataframes
df_A_CWP_1 = generate_DF_CW1(df_comb['106FT02A_CWP'], 'A_CWP_1')
df_A_CWP_2 = generate_DF_CW2(df_comb['106FT02A_CWP'], 'A_CWP_2')
df_A_N2DRY = generate_DF_for_N2DRY(df_comb['A_t_DRY'], 'A_N2DRY')
df_A_MANFP = generate_DF_comm(df_comb['106FT02A_MANFP'], 'A_MANF_P')
df_A_t_FEED = generate_DF_comm(df_comb['A_t_FEED'], 'A_FILT_T')
df_A_t_CWSH1 = generate_DF_comm(df_comb['A_t_CWSH1'], 'A_CWSH1')
df_A_t_CWSH2 = generate_DF_comm(df_comb['A_t_CWSH2'], 'A_CWSH2')
df_A_t_PRESS1 = generate_DF_comm(df_comb['A_t_PRESS1'], 'A_t_PRESS1')
df_A_t_PRESS2 = generate_DF_comm(df_comb['A_t_PRESS2'], 'A_t_PRESS2')
df_PU26A_OP = generate_DF_comm(df_comb['PU26A_OP'], 'A_PU26A_OP')
df_PU27A_OP = generate_DF_comm(df_comb['PU27A_OP'], 'A_PU27A_OP')
df_106PU26A_CUR = generate_DF_comm(df_comb['106PU26A_CUR'], 'A_PU26A_CUR')
df_106PU27A_CUR = generate_DF_comm(df_comb['106PU27A_CUR'], 'A_PU27A_CUR')
#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())

Done. Generated dataframes of each parameter for pressure filter alpha. 2020-02-03 16:11:15.120841


In [356]:
dfn = df_A_CWP_1.join(df_A_CWP_2, how='outer')
dfn1 = df_A_N2DRY.join(df_A_MANFP, how='outer')
dfn2 = df_A_t_FEED.join(df_A_t_CWSH1, how='outer')
dfn3 = df_A_t_CWSH2.join(df_A_t_PRESS1, how='outer')
dfn4 = df_A_t_PRESS2.join(df_PU26A_OP, how='outer')
dfn5 = df_PU27A_OP.join(df_106PU26A_CUR, how='outer')
dfn6 = df_106PU27A_CUR.join(dfn, how='outer')
dfn7 = dfn1.join(dfn2, how='outer')
dfn8 = dfn3.join(dfn4, how='outer')
dfn9 = dfn5.join(dfn6, how='outer')
dfn10 = dfn7.join(dfn8, how='outer')
dfn11 = dfn9.join(dfn10, how='outer')

df_PressureFilter_Param_A = dfn11

#-----
print('Done. Generated the merged dataframes of PiData parameters for pressure filter alpha.', datetime.datetime.now())

Done. Generated the merged dataframes of PiData parameters for pressure filter alpha. 2020-02-03 16:11:15.136771


# SELECT THE BRAVO PARAMETERS

# Clean the Data Using the functions

In [357]:
#Get the local min, max. Then generate the keys
ft_cyc_limB = getTheLocalMinMax(df_comb['B_t_FEED'], df_comb['B_t_DRY'])
keyslistB = list(ft_cyc_limB.keys())
#------
print('Done. Filtration Cycle Time generated.', datetime.datetime.now())


Done
Done. Filtration Cycle Time generated. 2020-02-03 16:11:15.668418


In [358]:
#Generate the dataframes for B
df_B_CWP_1 = generate_dfB_CW1(df_comb['FT02B_CWP'], 'B_CWP_1')
df_B_CWP_2 = generate_dfB_CW2(df_comb['FT02B_CWP'], 'B_CWP_2')
df_B_N2DRY = generate_dfB_for_N2DRY(df_comb['B_t_DRY'], 'B_N2DRY')
df_B_MANFP = generate_dfB_comm(df_comb['FT02B_MANFP'], 'B_MANF_P')
df_B_t_FEED = generate_dfB_comm(df_comb['B_t_FEED'], 'B_FILT_T')
df_B_t_CWSH1 = generate_dfB_comm(df_comb['B_t_CWSH1'], 'B_CWSH1')
df_B_t_CWSH2 = generate_dfB_comm(df_comb['B_t_CWSH2'], 'B_CWSH2')
df_B_t_PRESS1 = generate_dfB_comm(df_comb['B_t_PRESS1'], 'B_t_PRESS1')
df_B_t_PRESS2 = generate_dfB_comm(df_comb['B_t_PRESS2'], 'B_t_PRESS2')
df_PU26B_OP = generate_dfB_comm(df_comb['PU26B_OP'], 'B_PU26B_OP')
df_PU27B_OP = generate_dfB_comm(df_comb['PU27B_OP'], 'B_PU27B_OP')
df_106PU26B_CUR = generate_dfB_comm(df_comb['106PU26B_CUR'], 'B_PU26B_CUR')
df_106PU27B_CUR = generate_dfB_comm(df_comb['106PU27B_CUR'], 'B_PU27B_CUR')

#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())

Done. Generated dataframes of each parameter for pressure filter alpha. 2020-02-03 16:11:18.487797


In [359]:
#Combining the dataframes for bravo parameters
dfnB = df_B_CWP_1.join(df_B_CWP_2, how='outer')
dfn1B = df_B_N2DRY.join(df_B_MANFP, how='outer')
dfn2B = df_B_t_FEED.join(df_B_t_CWSH1, how='outer')
dfn3B = df_B_t_CWSH2.join(df_B_t_PRESS1, how='outer')
dfn4B = df_B_t_PRESS2.join(df_PU26B_OP, how='outer')
dfn5B = df_PU27B_OP.join(df_106PU26B_CUR, how='outer')
dfn6B = df_106PU27B_CUR.join(dfnB, how='outer')
dfn7B = dfn1B.join(dfn2B, how='outer')
dfn8B = dfn3B.join(dfn4B, how='outer')
dfn9B = dfn5B.join(dfn6B, how='outer')
dfn10B = dfn7B.join(dfn8B, how='outer')
dfn11B = dfn9B.join(dfn10B, how='outer')
df_PressureFilter_Param_B = dfn11B

#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())

Done. Generated dataframes of each parameter for pressure filter alpha. 2020-02-03 16:11:18.504756


# LABORATORY ANALYSIS RESULTS

In [360]:
#-----------------------
#collectst the index for the date.day the sample was taken and puts it in the list Smpl_day_index
def get_analysis_samplingDate_indexlst(sheetname, columnletter):

    Smpl_day_index = list()
    Smpl_day_index[:] = []
    for i in range (1, sheetname.max_row):
        if type(sheetname[columnletter+ str(i)].value) == int:
            Smpl_day_index.append(i)

    #smpl_end_index is the last row index for the date.day the sample was taken
    for i in range(1, sheetname.max_row):
        if str(sheetname[columnletter+str(i)].value).split(' ')[0] == 'Daily':
            smpl_end = i
            break
    return (Smpl_day_index, smpl_end)

#--------------------------------------------------
print('Done.', datetime.datetime.now())

Smpl_day_index = get_analysis_samplingDate_indexlst(FT02wb, "B")[0]
smpl_end_index = get_analysis_samplingDate_indexlst(FT02wb, "B")[1]
#-------------------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:11:18.622588
Done. 2020-02-03 16:11:18.641534


In [370]:
#--------------------------
#updated to treat datetime.datetime type entries, determine if the time entry is for the next day which is the case for first shift samples
def generate_element_df(sheetname, elmnt_col_lttr, df_elementNameA, df_elementNameB):

    #Code for creating a dataframe for MS_A analysis result for Element Pb
    A_Pb_datelist = list()
    A_Pb_datelist[:] = []
    B_Pb_datelist = list()
    B_Pb_datelist[:] = []

    A_Pb_list = list()
    A_Pb_list[:] = []
    B_Pb_list = list()
    B_Pb_list[:] = []

    A_Pb_list_pair = list()
    A_Pb_list_pair[:] = []
    B_Pb_list_pair = list()
    B_Pb_list_pair[:] = []

    for y in range(0, len(Smpl_day_index)):   

        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if sheetname['D'+str(x)].value != None:    
                    if filter_detrmin(str(sheetname['C'+str(x)].value)) == 'A':  

                        if type(sheetname['D'+str(x)].value) == datetime.datetime:
                            dt = sheetname['D'+str(x)].value
                            nt = str(dt.hour)+':'+str(dt.minute)+':'+str(dt.second)
                            A_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(nt))
                        else:    
                            A_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(sheetname['D'+str(x)].value))) 
                        
                        check_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str('7')+':'+str('00')+':'+str('00')))
         
                        for i in A_Pb_datelist:
                            if A_date < i and A_date < check_date:
                                A_date = A_date + datetime.timedelta(1)
                                               
                        A_Pb_datelist.append(A_date) 
                        A_Pb = sheetname[elmnt_col_lttr+str(x)].value
                        A_Pb_list.append(A_Pb)
                        
                    if filter_detrmin(str(sheetname['C'+str(x)].value)) == 'B':
                        
                        if type(sheetname['D'+str(x)].value) == datetime.datetime:
                            dt = sheetname['D'+str(x)].value
                            nt = str(dt.hour)+':'+str(dt.minute)+':'+str(dt.second)
                            B_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(nt))
                        else:    
                            B_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(sheetname['D'+str(x)].value)))
                                                
                        check_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str('7')+':'+str('00')+':'+str('00')))
         
                        for i in B_Pb_datelist:
                            if B_date < i and B_date < check_date:
                                B_date = B_date + datetime.timedelta(1)
                        
                        B_Pb_datelist.append(B_date)
                        B_Pb = sheetname[elmnt_col_lttr+str(x)].value
                        B_Pb_list.append(B_Pb)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index):
                if sheetname['D'+str(x)].value != None:    
                    if filter_detrmin(str(sheetname['C'+str(x)].value)) == 'A':
                                                                                              
                        if type(sheetname['D'+str(x)].value) == datetime.datetime:
                            dt = sheetname['D'+str(x)].value
                            nt = str(dt.hour)+':'+str(dt.minute)+':'+str(dt.second)
                            A_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(nt))
                        else:    
                            A_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(sheetname['D'+str(x)].value)))
                        
                        check_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str('7')+':'+str('00')+':'+str('00')))

                        for i in A_Pb_datelist:
                            if A_date < i and A_date < check_date:
                                A_date = A_date + datetime.timedelta(1)
                        
                        A_Pb_datelist.append(A_date)
                        A_Pb = sheetname[elmnt_col_lttr+str(x)].value
                        A_Pb_list.append(A_Pb)

                    if filter_detrmin(str(sheetname['C'+str(x)].value)) == 'B':
                        
                        if type(sheetname['D'+str(x)].value) == datetime.datetime:
                            dt = sheetname['D'+str(x)].value
                            nt = str(dt.hour)+':'+str(dt.minute)+':'+str(dt.second)
                            B_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(nt))
                        else:    
                            B_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(sheetname['D'+str(x)].value)))
                                                
                        check_date = parse(str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(sheetname['B'+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str('7')+':'+str('00')+':'+str('00')))
                        
                        for i in B_Pb_datelist:
                            if B_date < i and B_date < check_date:
                                B_date = B_date + datetime.timedelta(1)
                                               
                        B_Pb_datelist.append(B_date)
                        B_Pb = sheetname[elmnt_col_lttr+str(x)].value
                        B_Pb_list.append(B_Pb)
                        
    #this part creates the DataFrame after creating a list pair

    A_Pb_list_pair = list(zip(A_Pb_list, A_Pb_datelist[0:]))
    A_Pb_df = pd.DataFrame(A_Pb_list_pair)
    A_Pb_df.columns = [df_elementNameA, 'DATETIME'] ##check if list title is updated
    A_Pb_df.set_index('DATETIME', inplace=True)
    A_Element_df = A_Pb_df.loc[~A_Pb_df.index.duplicated(keep = 'last')]

    B_Pb_list_pair = list(zip(B_Pb_list, B_Pb_datelist[0:]))
    B_Pb_df = pd.DataFrame(B_Pb_list_pair)
    B_Pb_df.columns = [df_elementNameB, 'DATETIME'] ##check if list title is updated

    B_Pb_df.set_index('DATETIME', inplace=True)
    B_Element_df = B_Pb_df.loc[~B_Pb_df.index.duplicated(keep = 'last')]

    return (A_Element_df, B_Element_df)
#----------------------------------------------------------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:30:23.265722


In [371]:
#Generate dataframes using the generate_element_df function. [0] index for alpha, [1] index for bravo
df_Pb = generate_element_df(FT02wb, 'T', 'MS_A_Pb', 'MS_B_Pb')
df_Zn = generate_element_df(FT02wb, 'U', 'MS_A_Zn', 'MS_B_Zn')
df_Cu = generate_element_df(FT02wb, 'V', 'MS_A_Cu', 'MS_B_Cu')
df_Ni = generate_element_df(FT02wb, 'W', 'MS_A_Ni', 'MS_B_Ni')
df_Co = generate_element_df(FT02wb, 'X', 'MS_A_Co', 'MS_B_Co')
df_Fe = generate_element_df(FT02wb, 'Y', 'MS_A_Fe', 'MS_B_Fe')
df_Mn = generate_element_df(FT02wb, 'Z', 'MS_A_Mn', 'MS_B_Mn')
df_Cr = generate_element_df(FT02wb, 'AA', 'MS_A_Cr', 'MS_B_Cr')
df_Ca = generate_element_df(FT02wb, 'AB', 'MS_A_Ca', 'MS_B_Ca')
df_Si = generate_element_df(FT02wb, 'AC', 'MS_A_Si', 'MS_B_Si')
df_Al = generate_element_df(FT02wb, 'AD', 'MS_A_Al', 'MS_B_Al')
df_Mg = generate_element_df(FT02wb, 'AE', 'MS_A_Mg', 'MS_B_Mg')
df_S = generate_element_df(FT02wb, 'AF', 'MS_A_S', 'MS_B_S')

df_moisture = generate_element_df(FT02wb, 'AP', 'MS_A_moisture', 'MS_B_moisture')
df_PS_10D_spcl = generate_element_df(FT02wb, 'AQ', 'MS_A_PS_10D_spcl', 'MS_B_PS_10D_spcl')
df_PS_50D_spcl = generate_element_df(FT02wb, 'AR', 'MS_A_PS_50D_spcl', 'MS_B_PS_50D_spcl')
df_PS_90D_spcl = generate_element_df(FT02wb, 'AS', 'MS_A_PS_90D_spcl', 'MS_B_PS_90D_spcl')
df_PS_mean_spcl = generate_element_df(FT02wb, 'AT', 'MS_A_PS_mean_v_spcl', 'MS_A_PS_mean_v_spcl')
df_PS_stdev_spcl = generate_element_df(FT02wb, 'AU', 'MS_B_PS_stdev_spcl', 'MS_B_PS_stdev_spcl')

#-----------------------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:30:39.171115


In [372]:
#Pressure Filter A lab results in merged dataframe
df_A_PbZn = df_Pb[0].join(df_Zn[0], how='outer')
df_A_CuNi = df_Cu[0].join(df_Ni[0], how='outer')
df_A_CoFe = df_Co[0].join(df_Fe[0], how='outer')
df_A_MnCr = df_Mn[0].join(df_Cr[0], how='outer')
df_A_CaSi = df_Ca[0].join(df_Si[0], how='outer')
df_A_AlMg = df_Al[0].join(df_Mg[0], how='outer')
df_A_Sh2o = df_S[0].join(df_moisture[0], how='outer')
df_A_PS_10D50D = df_PS_10D_spcl[0].join(df_PS_50D_spcl[0], how='outer')
df_A_PS_90Dmean = df_PS_90D_spcl[0].join(df_PS_mean_spcl[0], how='outer')

df_A_PbZnCuNi = df_A_PbZn.join(df_A_CuNi, how='outer')
df_A_CoFeMnCr = df_A_CoFe.join(df_A_MnCr, how='outer')
df_A_CaSiAlMg = df_A_CaSi.join(df_A_AlMg, how='outer')
df_A_Sh2oPS10D50D = df_A_Sh2o.join(df_A_PS_10D50D, how='outer')
df_A_PS_90Dmeanstdev = df_A_PS_90Dmean.join(df_PS_stdev_spcl[0], how='outer')

df_A_PbZnCuNiCoFeMnCr = df_A_PbZnCuNi.join(df_A_CoFeMnCr, how='outer')
df_A_CaSiAlMgSh2oPS10D50D = df_A_CaSiAlMg.join(df_A_Sh2oPS10D50D, how='outer')
df_A_CaSiAlMgSh2oPS10D50D90Dmeanstdev = df_A_CaSiAlMgSh2oPS10D50D.join(df_A_PS_90Dmeanstdev, how='outer')
df_A_PbZnCuNiCoFeMnCrCaSiAlMgSh2oPS10D50D90Dmeanstdev = df_A_PbZnCuNiCoFeMnCr.join(df_A_CaSiAlMgSh2oPS10D50D90Dmeanstdev, how='outer')
df_A_merged_lab_res = df_A_PbZnCuNiCoFeMnCrCaSiAlMgSh2oPS10D50D90Dmeanstdev

#-----------------------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:31:07.966988


In [373]:
df_B_PbZn = df_Pb[1].join(df_Zn[1], how='outer')
df_B_CuNi = df_Cu[1].join(df_Ni[1], how='outer')
df_B_CoFe = df_Co[1].join(df_Fe[1], how='outer')
df_B_MnCr = df_Mn[1].join(df_Cr[1], how='outer')
df_B_CaSi = df_Ca[1].join(df_Si[1], how='outer')
df_B_AlMg = df_Al[1].join(df_Mg[1], how='outer')
df_B_Sh2o = df_S[1].join(df_moisture[1], how='outer')
df_B_PS_10D50D = df_PS_10D_spcl[1].join(df_PS_50D_spcl[1], how='outer')
df_B_PS_90Dmean = df_PS_90D_spcl[1].join(df_PS_mean_spcl[1], how='outer')

df_B_PS_90Dmeanstdev = df_B_PS_90Dmean.join(df_PS_stdev_spcl[1], how='outer')
df_B_PbZnCuNi = df_B_PbZn.join(df_B_CuNi, how='outer')
df_B_CoFeMnCr = df_B_CoFe.join(df_B_MnCr, how='outer')
df_B_CaSiAlMg = df_B_CaSi.join(df_B_AlMg, how='outer')
df_B_Sh2o9D50D = df_B_Sh2o.join(df_B_PS_10D50D, how='outer')

df_B_PbZnCuNiCoFeMnCr = df_B_PbZnCuNi.join(df_B_CoFeMnCr, how='outer')
df_B_CaSiAlMgSh2o9D50D = df_B_CaSiAlMg.join(df_B_Sh2o9D50D, how='outer')

df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D = df_B_PbZnCuNiCoFeMnCr.join(df_B_CaSiAlMgSh2o9D50D, how='outer')
df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D90Dmeanstdev = df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D.join(df_B_PS_90Dmeanstdev, how='outer')

df_B_merged_lab_res = df_B_PbZnCuNiCoFeMnCrCaSiAlMgSh2o9D50D90Dmeanstdev

#deletes the rows whose index has duplicates but keeps the last value
df_A_merged_lab_res = df_A_merged_lab_res.loc[~df_A_merged_lab_res.index.duplicated(keep='last')]
df_B_merged_lab_res = df_B_merged_lab_res.loc[~df_B_merged_lab_res.index.duplicated(keep='last')]

df_A_merged_lab_res.sort_index(inplace=True)
df_B_merged_lab_res.sort_index(inplace=True)

#--------------------
print('Done', datetime.datetime.now())

Done 2020-02-03 16:31:08.162291


In [374]:
#---------Create new columns that separates the daterange written in string format then parses it to actual datetime
df_PressureFilter_Param_A['newdt'] = df_PressureFilter_Param_A.index
df_PressureFilter_Param_A['startdate'] = df_PressureFilter_Param_A['newdt'].apply(lambda x: datesplitter_start(x))
df_PressureFilter_Param_A['enddate'] = df_PressureFilter_Param_A['newdt'].apply(lambda x: datesplitter_end(x))

df_PressureFilter_Param_B['newdt'] = df_PressureFilter_Param_B.index
df_PressureFilter_Param_B['startdate'] = df_PressureFilter_Param_B['newdt'].apply(lambda x: datesplitter_start(x))
df_PressureFilter_Param_B['enddate'] = df_PressureFilter_Param_B['newdt'].apply(lambda x: datesplitter_end(x))

#---------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:31:08.488494


In [375]:
def generate_df(lst_index, lst_value, colname_output):
    lst_index_batch_comp_wt = list(zip(lst_index,lst_value))
    df_batch_comp_wt_index_time = pd.DataFrame(lst_index_batch_comp_wt)

    df_batch_comp_wt_index_time.columns = ['DATETIME', colname_output]
    df_bc_wt = df_batch_comp_wt_index_time
    df_bc_wt.set_index('DATETIME', inplace= True)
    
    return df_bc_wt
#---------------------
def generate_cycle_time(df_of_parameter, main_df_containing_datetimerange):
    w_n11_indexList = []
    w_n11_bcwt_indexList = []

    for i in range(0, len(df_of_parameter)):
        for j in range(0, len(main_df_containing_datetimerange)):
            if j < len(main_df_containing_datetimerange):
                if main_df_containing_datetimerange['enddate'][j-1] <= df_of_parameter.index[i] and main_df_containing_datetimerange['enddate'][j] >= df_of_parameter.index[i]:
                    w_n11_indexList.append(main_df_containing_datetimerange.index[j-1])
                    w_n11_bcwt_indexList.append(df_of_parameter.index[i]) 

    df_parameter_cycTime = generate_df(w_n11_bcwt_indexList, w_n11_indexList[0:], 'FT_cycletimerange')
    return df_parameter_cycTime

#--------------------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:31:08.909825


# COMBINE PiDATA WITH LABORATORY RESULTS FOR ALPHA

In [376]:
#code for changing the index to Datetime in range format (from date1 to date2)

A_column_names_laboratory_results = ['MS_A_Pb_spcl', 'MS_A_Zn_spcl', 'MS_A_Cu_spcl', 'MS_A_Ni_spcl', 'MS_A_Co_spcl', 'MS_A_Fe_spcl',
       'MS_A_Mn_spcl', 'MS_A_Cr_spcl', 'MS_A_Ca_spcl', 'MS_A_Si_spcl', 'MS_A_Al_spcl', 'MS_A_Mg_spcl','MS_A_S_spcl', 'MS_A_moisture_spcl', 'MS_A_PS_10D_spcl', 'MS_A_PS_50D_spcl', 'MS_A_PS_90D_spcl',
       'MS_A_PS_mean_v_spcl', 'MS_A_PS_stdev_spcl','DATETIME']

A_column_names_merged_final = ['Cycle_Count','A_PU26A_CUR', 'A_PU27A_CUR', 'A_N2DRY', 'A_CWP_1', 'A_CWP_2',
       'A_FILT_T', 'A_MANF_P', 'A_CWSH1', 'A_CWSH2', 'A_t_PRESS1',
       'A_t_PRESS2', 'A_PU26A_OP', 'A_PU27A_OP', 'newdt', 'startdate',
       'enddate', 'MS_A_Pb_spcl', 'MS_A_Zn_spcl', 'MS_A_Cu_spcl',
       'MS_A_Ni_spcl', 'MS_A_Co_spcl', 'MS_A_Fe_spcl', 'MS_A_Mn_spcl',
       'MS_A_Cr_spcl', 'MS_A_Ca_spcl', 'MS_A_Si_spcl', 'MS_A_Al_spcl',
       'MS_A_Mg_spcl', 'MS_A_S_spcl', 'MS_A_moisture_spcl', 'MS_A_PS_10D_spcl',
       'MS_A_PS_50D_spcl', 'MS_A_PS_90D_spcl', 'MS_A_PS_mean_v_spcl',
       'MS_A_PS_stdev_spcl']

dfEL4 = generate_cycle_time(df_A_merged_lab_res, df_PressureFilter_Param_A)
dfEL5 = df_A_merged_lab_res.join(dfEL4, how='outer')
dfEL5.columns = A_column_names_laboratory_results #Rename the column using the prepared list
dfEL5.set_index('DATETIME', inplace=True)

df_Cycle_LabRes = dfn11.join(dfEL5, how='outer')
df_Cycle_LabRes.fillna(method='ffill')
df_Cycle_LabRes['Cycle_Count'] = np.arange(1, len(df_Cycle_LabRes['A_FILT_T'])+1)
df_Cycle_LabRes_2 = df_Cycle_LabRes.reindex(columns= A_column_names_merged_final)
df_Cycle_LabRes_2_ffillna = df_Cycle_LabRes_2.fillna(method='ffill')

A_df_JAN2020_cycLab = df_Cycle_LabRes_2_ffillna

#---------dfEL5 is the is the analysis results data indexed by daterange format. readt for merging with dfn11(FT02A data from Pi)
#---------
print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:31:11.294452


# COMBINE PiDATA WITH LABORATORY RESULTS FOR BRAVO

In [377]:
#---------
B_column_names_laboratory_results = ['MS_B_Pb_spcl', 'MS_B_Zn_spcl', 'MS_B_Cu_spcl', 'MS_B_Ni_spcl', 'MS_B_Co_spcl', 'MS_B_Fe_spcl',
       'MS_B_Mn_spcl', 'MS_B_Cr_spcl', 'MS_B_Ca_spcl', 'MS_B_Si_spcl', 'MS_B_Al_spcl', 'MS_B_Mg_spcl','MS_B_S_spcl', 'MS_B_moisture_spcl', 'MS_B_PS_10D_spcl', 'MS_B_PS_50D_spcl', 'MS_B_PS_90D_spcl',
       'MS_B_PS_mean_v_spcl', 'MS_B_PS_stdev_spcl','DATETIME']

B_column_names_merged_final = ['Cycle_Count','B_PU26B_CUR', 'B_PU27B_CUR', 'B_N2DRY', 'B_CWP_1', 'B_CWP_2',
       'B_FILT_T', 'B_MANF_P', 'B_CWSH1', 'B_CWSH2', 'B_t_PRESS1',
       'B_t_PRESS2', 'B_PU26B_OP', 'B_PU27B_OP', 'newdt', 'startdate',
       'enddate', 'MS_B_Pb_spcl', 'MS_B_Zn_spcl', 'MS_B_Cu_spcl',
       'MS_B_Ni_spcl', 'MS_B_Co_spcl', 'MS_B_Fe_spcl', 'MS_B_Mn_spcl',
       'MS_B_Cr_spcl', 'MS_B_Ca_spcl', 'MS_B_Si_spcl', 'MS_B_Al_spcl',
       'MS_B_Mg_spcl', 'MS_B_S_spcl', 'MS_B_moisture_spcl', 'MS_B_PS_10D_spcl',
       'MS_B_PS_50D_spcl', 'MS_B_PS_90D_spcl', 'MS_B_PS_mean_v_spcl',
       'MS_B_PS_stdev_spcl']

B_Lab_Res_formatted_index = generate_cycle_time(df_B_merged_lab_res, df_PressureFilter_Param_B) #the function creates an index in datetime range in string format the same with the index format of the main_df (dfn11 or dfn11B)

B_df_merged_lab_res_fomatted_index = df_B_merged_lab_res.join(B_Lab_Res_formatted_index, how='outer')
B_df_merged_lab_res_fomatted_index.columns = B_column_names_laboratory_results #Rename the column names

B_df_merged_lab_res_fomatted_index.set_index('DATETIME', inplace=True)

df_Cycle_LabResB = dfn11B.join(B_df_merged_lab_res_fomatted_index, how='outer')
df_Cycle_LabResB.fillna(method='ffill')
df_Cycle_LabResB['Cycle_Count'] = np.arange(1, len(df_Cycle_LabResB['B_FILT_T'])+1)
df_Cycle_LabRes_2B = df_Cycle_LabResB.reindex(columns= B_column_names_merged_final)
df_Cycle_LabRes_2B_ffillna = df_Cycle_LabRes_2B.fillna(method='ffill')

B_df_JAN2020_Combined_LabRes_and_Filtration_Param = df_Cycle_LabRes_2B_ffillna

#---------------B_df_merged_lab_res_fomatted_index is the is the analysis results data indexed by daterange format. readt for merging with dfn11(FT02B data from Pi)

print('Done.', datetime.datetime.now())

Done. 2020-02-03 16:31:12.777324


# COMBINE PiDATA WITH LABORATORY RESULTS FOR BRAVO

# SAVE THE DATAFRAME TO EXCEL

In [378]:
A_df_JAN2020_cycLab.to_excel(r'C:\Users\v.t.flores\Documents\2019_PF_Data_Rev2\106FT02A_data_DEC2019.xlsx')
#A_df_JAN2020_cycLab.to_excel(r'C:\Users\v.t.flores\Documents\106FT02A_data_JAN2019.xlsx')
#A_df_JAN2020_cycLab.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Pressure_Filter_Data_2019\106FT02A_data_JAN2020.xlsx')

In [379]:
B_df_JAN2020_Combined_LabRes_and_Filtration_Param.to_excel(r'C:\Users\v.t.flores\Documents\2019_PF_Data_Rev2\106FT02B_data_DEC2019.xlsx')
#B_df_JAN2020_Combined_LabRes_and_Filtration_Param.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Pressure_Filter_Data_2019\106FT02B_data_JAN2020.xlsx')